In [16]:
import os

import pandas as pd

print( os.getcwd() )
# change working directory
os.chdir( "/var/genie-in-the-box/src" )
print( os.getcwd() )
import lib.utils.util as du

/var/genie-in-the-box/src
/var/genie-in-the-box/src


In [2]:
# associate commands with files containing examples
browser_commands = {
    "search new tab": "/src/ephemera/prompts/data/synthetic-data-search-in-new-tab.txt",
    "search current tab": "/src/ephemera/prompts/data/synthetic-data-search-in-current-tab.txt",
    "search google new tab": "/src/ephemera/prompts/data/synthetic-data-search-google-in-new-tab.txt",
    "search google current tab": "/src/ephemera/prompts/data/synthetic-data-search-google-in-current-tab.txt",
    "search google scholar new tab": "/src/ephemera/prompts/data/synthetic-data-search-google-scholar-in-new-tab.txt",
    "search google scholar current tab": "/src/ephemera/prompts/data/synthetic-data-search-google-scholar-in-current-tab.txt",
}

In [3]:
for command in browser_commands.keys():
    print( os.path.exists( du.get_project_root() + browser_commands[ command ] ) )

True
True
True
True
True
True


In [4]:
raw_lines = du.get_file_as_list( du.get_project_root() + "/src/ephemera/prompts/data/synthetic-data-search-in-new-tab.txt", clean=True )[ 0:100 ]
raw_lines[ 0:10 ]

['SEARCH_TERMS new tab search',
 'New tab, search SEARCH_TERMS',
 'Open new tab, search for SEARCH_TERMS',
 'Pull up SEARCH_TERMS in another tab',
 'Find SEARCH_TERMS in a new tab',
 'Hunt for SEARCH_TERMS, new tab',
 'Display SEARCH_TERMS results in new tab',
 'Do a search for SEARCH_TERMS and show results in another tab',
 'Execute SEARCH_TERMS search in a new tab',
 'Seek SEARCH_TERMS in a fresh tab']

In [5]:
def get_search_terms( requested_length ):
    
    # Load search terms file
    search_terms = du.get_file_as_list( du.get_project_root() + "/src/ephemera/prompts/data/search-terms.txt", lower_case=False, clean=True, randomize=True )
    
    # If we don't have enough search terms, append copies of the search term list until we do
    while requested_length > len( search_terms ):
        search_terms += search_terms
        
    # Truncate the search terms list to equal the requested len
    search_terms = search_terms[ :requested_length ]
    
    return search_terms

In [6]:
search_terms = get_search_terms( len( raw_lines ) )
search_terms[ 0:10]

['Using Pandas in data science projects',
 'DIY home decor ideas',
 'weather forecast washington dc',
 'TensorFlow latest updates',
 'Data imputation with Pandas',
 'Google Cloud AI services',
 'AI for personalized education',
 'how to overcome anxiety',
 'AI in autonomous vehicles',
 'Reinforcement Learning breakthroughs']

In [7]:
command_choices = "'" + "', '".join( browser_commands.keys() ) + "' and 'none'"
command_choices 

"'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'"

In [8]:
command_choices = "'" + "', '".join( browser_commands.keys() ) + "' and 'none'"
question_template = """
Your job is to discern the intent of a transcribed human voice command and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `{command_choices}`.

Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

Below is the raw human voice command transcription formatted using simple XML: 
{human_says}
The standardized command that you translate MUST be returned wrapped in simple, well-formed XML:
{response_format}
"""

human_says_template = """
<human>
    <voice-command>{voice_command}</voice-command>
</human>
"""

response_format = """
<response>
    <browser-command></browser-command>
    <args></args>
</response>
"""

answer_template = """
<response>
    <browser-command>{browser_command}</browser-command>
    <args>{args}</args>
</response>
"""

In [10]:
for line in question_template.split( "\n" ):
    print( line )


Your job is to discern the intent of a transcribed human voice command and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `{command_choices}`.

Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

Below is the raw human voice command transcription formatted using simple XML: 
{human_says}
The standardized command that you translate MUST be returned wrapped in simple, well-formed XML:
{response_format}


In [11]:
questions   = []
answers   = []
responses = []

for browser_command in browser_commands.keys():
    
    du.print_banner( browser_command, prepend_nl=True, end="\n" )
    
    raw_lines = du.get_file_as_list( du.get_project_root() + browser_commands[ browser_command ], clean=True )
    for line in raw_lines[ 0:100 ]: #[ 0:2 ]:#
        
        # get newly randomized search terms on every iteration
        for search in get_search_terms( len( raw_lines ) ):#[ 0:10 ]: #[ 0:2]:#:
            
            voice_command = line.replace( "SEARCH_TERMS", search )
            # print( voice_command )
            human_says    = human_says_template.format( voice_command=voice_command )
            question      = question_template.format( human_says=human_says, response_format=response_format, command_choices=command_choices )
            answer        = answer_template.format( browser_command=browser_command, args=search )
    
            questions.append( question )
            answers.append( answer )
    
            # du.print_banner( "Prompt" )
            # print( prompt )
            # du.print_banner( "Response" )
            # print( response )
            print( ".", end="" )
        
    print()


------------------------------------------------------------------------------------------------------------------------
- search new tab
------------------------------------------------------------------------------------------------------------------------
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [12]:
idx = 0

for line in questions[ 0 ].split( "\n" ):
    print( line )
    
for line in answers[ idx ].split( "\n" ):
    print( line )


Your job is to discern the intent of a transcribed human voice command and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

Below is the raw human voice command transcription formatted using simple XML: 

<human>
    <voice-command>best hiking trails near me new tab search</voice-command>
</human>

The standardized command that you translate MUST be returned wrapped in simple, well-formed XML:

<response>
    <browser-command></browser-command>
    <args></args>
</response>



<response>
    <browser-c

In [13]:
len( answers )

120000

In [14]:
len( questions )

120000

In [109]:
for line in questions[ 0 ].split( "\n" ):
    print( line )


Your job is to discern the intent of a transcribed human voice command and translate it into a standardized command that a browser on your computer would understand.

You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

The raw human voice command transcription is in simple XML: 

<human>
    <voice-command>I need hiking gear new tab search</voice-command>
</human>

The standardized command that you translate MUST be returned in simple, well-formed XML:

<response>
    <browser-command></browser-command>
    <args></args>
</response>


In [86]:
for line in answers[ 0 ].split( "\n" ):
    print( line )


<response>
    <browser-command>search new tab</browser-command>
    <args>why onions make us cry</args>
</response>


In [20]:
import random

def inject_space( line ):
    if random.random() < 0.5:
        index = random.randint( 0, len( line ) - 1 )
        return line[ :index ] + ' ' + line[ index: ]
    else:
        return line

In [22]:
qna_df = pd.DataFrame( { "question": questions, "answer": answers } )

# Perturb the answer to create a faked response  
qna_df[ "response" ] = qna_df[ "answer" ].apply( lambda cell: inject_space( cell ) )
qna_df.head()

,question,answer,response
0,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-co mmand>search new...
1,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...
2,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new...
3,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...
4,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...


## Validate the structure of the xml response

In [ ]:
!pip install xmlschema

In [30]:
from xmlschema import XMLSchema

xml_str_1 = """
<response>
  <browser-command></browser-command>
  <args></args>
</response>
"""
xml_str_2 = """
<response>
  <browser_command></browser_command>
  <args></args>
</response>
"""
xml_str_3 = """
Sure here's your command!
<response>
  <browser-command></browser-command>
  <args></args>
</response>
"""

xml_str_4 = """
```xml
<response>
  <browser-command></browser-command>
  <args></args>
</response>
"""

xsd_string = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="response">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="browser-command" type="xs:string"/>
        <xs:element name="args" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

def is_valid_xml( xml_str, schema ):
    
    try:
        schema.is_valid( xml_str )
        return True
    except Exception as e:
        # print( "Invalid XML: " + str( e ) )
        return False

schema   = XMLSchema( xsd_string )

print( is_valid_xml( xml_str_1, schema ) )  # Output: ???
print( is_valid_xml( xml_str_2, schema ) )  # Output: ???
print( is_valid_xml( xml_str_3, schema ) )  # Output: False
print( is_valid_xml( xml_str_4, schema ) )  # Output: False

True
True
False
False


In [31]:
qna_df[ "response_xml_is_valid" ] = qna_df[ "response" ].apply( lambda cell: is_valid_xml( cell, schema ) )
qna_df.head()

,question,answer,response,response_xml_is_valid
0,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-co mmand>search new...,False
1,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True
2,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new...,True
3,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True
4,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True


In [34]:
qna_df.response_xml_is_valid.mean()

0.7800916666666666

## Validate the response by removing all white space and comparing to the original answer

In [35]:
import re

xml_str = """
<response>
  <browser-command>Whos your favorite browser?</browser-command>
  <args>Bar browser</args>
</response>
"""

# Remove white space outside XML tags
xml_str = re.sub(r'>\s+<', '><', xml_str.strip())

print(xml_str)

<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>


In [39]:
def is_response_correct( response, answer ):
    # Remove white space outside XML tags
    response = re.sub( r'>\s+<', '><', response.strip() )
    
    # Remove white space outside XML tags
    answer = re.sub( r'>\s+<', '><', answer.strip() )
    
    return response == answer

is_response_correct( "<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>", "<response><browser-command>Whos your favorite browser?</browser-command><args>Bar browser</args></response>" )

True

In [40]:
qna_df[ "response_is_correct" ] = qna_df.apply( lambda row: is_response_correct( row[ "response" ], row[ "answer" ] ), axis=1 )

In [41]:
qna_df.response_is_correct.mean()

0.5654333333333333

In [43]:
qna_df[ ( qna_df.response_xml_is_valid == True ) & ( qna_df.response_is_correct == False ) ].head( 10 )

,question,answer,response,response_xml_is_valid,response_is_correct
3,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True,False
10,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new...,True,False
18,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True,False
26,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True,False
28,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True,False
30,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search ne w...,True,False
39,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True,False
47,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>s earch new...,True,False
48,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new ...,True,False
50,\nYour job is to discern the intent of a trans...,\n<response>\n <browser-command>search new ...,\n<response>\n <browser-command>search new...,True,False


# What's the baseline behavior of the Vanilla model?